In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/translate-school-json-teacher-json/__results__.html
/kaggle/input/translate-school-json-teacher-json/en-teacher.csv
/kaggle/input/translate-school-json-teacher-json/__notebook__.ipynb
/kaggle/input/translate-school-json-teacher-json/en-school.csv
/kaggle/input/translate-school-json-teacher-json/__output__.json
/kaggle/input/translate-school-json-teacher-json/custom.css


# 1. Download data:

In [2]:
school = pd.read_csv('/kaggle/input/translate-school-json-teacher-json/en-school.csv')
display(school.head())
display(school.describe())
teacher = pd.read_csv('/kaggle/input/translate-school-json-teacher-json/en-teacher.csv')
display(teacher.head())
display(teacher.describe())

,Unnamed: 0,id,name_translated,about_translated,motto_translated
0,0,S_1,Tsinghua University,"It is referred to as ""Tsinghua"" for short. It ...",Self-improvement and virtue carry things
1,1,S_2,Beijing University,"Peking University, referred to as ""Peking Univ...","Learn, interrogate, think carefully, and disti..."
2,2,S_3,Wuhan University,"Wuhan University (Wuhan University, abbreviate...","Self-reliance, Hongyi, Seek truth, and expand new"
3,3,S_4,Suzhou University,"Soochow University is located in Suzhou, a fam...",To nourish the righteousness of heaven and ear...
4,4,S_5,Sichuan University,"Sichuan University (Sichuan University), refer...",NaN


,Unnamed: 0
count,429.000000
mean,214.000000
std,123.985886
min,0.000000
25%,107.000000
50%,214.000000
75%,321.000000
max,428.000000


,Unnamed: 0,id,name_translated,about_translated,job_title_translated,org_name_translated
0,0,T_1,Liu Yanni,Graduated from the Department of Philosophy at...,lecturer,Tsinghua University
1,1,T_2,Chen Yi,"Born in Chongqing in 1945, he graduated from t...",professor,Tsinghua University
2,2,T_3,Cheng Gang,"Cheng Gang, the director of the course ""Introd...",Associate Professor,Tsinghua University
3,3,T_4,Xie Weihe,"Xie Weihe is a doctor, professor, doctoral sup...",professor,Tsinghua University
4,4,T_5,Shi Jinghuan,"Shi Jinghuan, female, professor and doctoral s...",professor,Tsinghua University


,Unnamed: 0
count,17018.000000
mean,8508.500000
std,4912.817776
min,0.000000
25%,4254.250000
50%,8508.500000
75%,12762.750000
max,17017.000000


# 2. Preprocessing:
- Translate
- Clean text

## Functions

In [3]:
!pip install deep-translator

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 1.7 MB/s eta 0:00:00


In [4]:
!pip install nltk==3.9b1

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 33.8 MB/s eta 0:00:00
  Attempting uninstall: nltk
    Found existing installation: nltk 3.2.4
    Uninstalling nltk-3.2.4:
      Successfully uninstalled nltk-3.2.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
preprocessing 0.1.13 requires nltk==3.2.4, but you have nltk 3.9b1 which is incompatible.


In [5]:
import re
import unicodedata
import time
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import zipfile
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('stopwords')
nltk.download('wordnet')
zip_file_path = '/usr/share/nltk_data/corpora/wordnet.zip'
extract_to_path = '/usr/share/nltk_data/corpora/'
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to_path)
lemmatizer = WordNetLemmatizer()

# 
from deep_translator import GoogleTranslator

def contains_chinese(text):
    return bool(re.search(r'[\u4e00-\u9fff]', str(text)))
    
def translate_if_chinese(text):
    if contains_chinese(text):
        try:
            time.sleep(1)
            return GoogleTranslator(source='zh-CN', target='en').translate(text)
        except Exception as e:
            print(f"Lỗi dịch: {e}")
            return text
    return text

def clean_text(text):
    if not isinstance(text, str):
        return ""
    # 1. Chuyển về chữ thường
    text = text.lower().strip()

    # 2. Chuẩn hóa Unicode
    text = unicodedata.normalize("NFKC", text)

    # 3. Loại bỏ số, dấu câu, ký tự đặc biệt (chỉ giữ lại chữ cái và khoảng trắng)
    text = re.sub(r"[^\w\s]", "", text)  # Xóa dấu câu
    text = re.sub(r"\d+", "", text)  # Xóa số

    # 4. Tokenization (Tách từ)
    words = word_tokenize(text)

    # 5. Loại bỏ stopwords
    stop_words = set(stopwords.words("english"))
    words = [word for word in words if word not in stop_words]

    # 6. Lemmatization (Chuyển về dạng gốc)
    words = [lemmatizer.lemmatize(word) for word in words]

    # 7. Gộp lại thành câu
    return " ".join(words)
    
def preprocess_text_column(df, column_name):  # Đảm bảo dữ liệu là chuỗi
    df[column_name] = df[column_name].apply(clean_text)  # Áp dụng hàm xử lý
    return df

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /usr/share/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [6]:
def checklang(df):
    cols = df.columns.tolist()
    cols.remove('id')
    cols.remove('Unnamed: 0')
    for col in cols:
        display(df[col][df[col].apply(contains_chinese)])
def preprocessing(df):
    cols = df.columns.tolist()
    cols.remove('id')
    cols.remove('Unnamed: 0')
    for col in cols:
        df = preprocess_text_column(df,col)
    return df

## Translate

In [7]:
school['name_translated'] = school['name_translated'].apply(translate_if_chinese)
school['about_translated'] = school['about_translated'].apply(translate_if_chinese)
school['motto_translated'] = school['motto_translated'].apply(translate_if_chinese)

In [8]:
teacher['name_translated'] = teacher['name_translated'].apply(translate_if_chinese)
teacher['about_translated'] = teacher['about_translated'].apply(translate_if_chinese)
teacher['job_title_translated'] = teacher['job_title_translated'].apply(translate_if_chinese)
teacher['org_name_translated'] = teacher['org_name_translated'].apply(translate_if_chinese)

Lỗi dịch: Request exception can happen due to an api connection error. Please check your connection and try again
Lỗi dịch: Request exception can happen due to an api connection error. Please check your connection and try again
Lỗi dịch: Request exception can happen due to an api connection error. Please check your connection and try again
Lỗi dịch: Request exception can happen due to an api connection error. Please check your connection and try again


In [9]:
checklang(school)

Series([], Name: name_translated, dtype: object)

Series([], Name: about_translated, dtype: object)

Series([], Name: motto_translated, dtype: object)

In [10]:
checklang(teacher)

2342    Danny丨Guest Tutor
2343    Henry丨Guest Tutor
Name: name_translated, dtype: object

4583     学历： 2016/02-2016/04  Toronto General Hospital,...
14264    巩如英\n电话：13513366770;  0335-7815919\nE-mail: ry...
14655    一、个人简介：\n    张怀钊，男，汉族，1973年9月生，宁夏西吉县人，1998年7月参...
15178            杨银科，男，副教授，博士，硕士研究生导师，中共党员，长安大学水利与环境学院水...
Name: about_translated, dtype: object

Series([], Name: job_title_translated, dtype: object)

Series([], Name: org_name_translated, dtype: object)

## Clean text

In [11]:
preprocess_school = preprocessing(school)

In [12]:
preprocess_teacher = preprocessing(teacher)

# Save file

In [13]:
preprocess_school.to_csv('school.csv')
preprocess_teacher.to_csv('teacher.csv')